In [ ]:
!pip install ../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0/

In [ ]:
import numpy as np 
import pandas as pd
import json
import os
import cv2
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

import warnings  
warnings.filterwarnings('ignore')

from distutils.dir_util import copy_tree
#model = '../input/resnet152/'
model = '../input/efficientnet-pytorch/'
checkpoints = '/root/.cache/torch/hub/checkpoints/'
copy_tree(model,checkpoints)

In [ ]:
BATCH = 5
EPOCHS = 1

LR = 0.0001
IM_SIZE = 512

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images/'
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'

In [ ]:
labels = json.load(open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"))
print(labels)

train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head()

In [ ]:
X_Train, Y_Train = train['image_id'].values, train['label'].values
X_Test = [name for name in (os.listdir(TEST_DIR))]

In [ ]:
#X_Train = X_Train[0:1050]
#Y_Train = Y_Train[0:1050]

In [ ]:
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        
        image = cv2.imread(os.path.join(self.dir, self.fnames[index]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        original = image
        new_color = image

        lower = np.array([60, 60, 120])
        upper = np.array([250, 120, 215])
        mask = cv2.inRange(new_color, lower, upper)
        result4 = cv2.bitwise_and(original,original,mask=mask)
        result4 = cv2.cvtColor(result4, cv2.COLOR_LAB2RGB)
        result4[mask==0] = (255,255,255)          
        
        lower = np.array([165, 115, 185])
        upper = np.array([225, 155, 210])
        mask = cv2.inRange(new_color, lower, upper)
        result2 = cv2.bitwise_and(original,original,mask=mask)
        result2 = cv2.cvtColor(result2, cv2.COLOR_LAB2RGB)
        result2[mask==0] = (255,255,255)

        lower = np.array([195, 105, 125])
        upper = np.array([255, 125, 145])
        mask = cv2.inRange(new_color, lower, upper)
        result3 = cv2.bitwise_and(original,original,mask=mask)
        result3 = cv2.cvtColor(result3, cv2.COLOR_LAB2RGB)
        result3[mask==0] = (255,255,255)

        lower = np.array([115, 118, 120])
        upper = np.array([217, 130, 130])
        mask = cv2.inRange(new_color, lower, upper)
        result5 = cv2.bitwise_and(original,original,mask=mask)
        result5 = cv2.cvtColor(result5, cv2.COLOR_LAB2RGB)
        result5[mask==0] = (255,255,255)

        result = cv2.bitwise_and(result4,result2)
        result = cv2.bitwise_and(result,result3)
        image = cv2.bitwise_and(result,result5)

        if "train" in self.dir:
            image = Transform_train(image=image)['image']
            return image, self.lbs[index]
        
        elif "test" in self.dir: 
            image = Transform_test(image=image)['image']
            return image, self.fnames[index]

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop,RandomGridShuffle, Resize
)
from albumentations.pytorch import ToTensorV2

In [ ]:
Transform_train = Compose([
            CenterCrop(IM_SIZE, IM_SIZE),
            RandomGridShuffle(grid=(2, 2), p=0.5),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            #HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            #RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            #CoarseDropout(p=0.5),
            #Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)

Transform_test = Compose([
            CenterCrop(IM_SIZE, IM_SIZE),
            #Resize(IM_SIZE, IM_SIZE),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=5)
model.load_state_dict(torch.load('../input/my-model2/my_model (2)'))

In [ ]:
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# from sklearn.model_selection import train_test_split

# x_train_idx, x_valid_idx, y_train_idx, y_valid_idx = train_test_split(X_Train,Y_Train, test_size=0.1,shuffle =True)
# print(x_train_idx.shape, y_train_idx.shape)
# print(x_valid_idx.shape, y_valid_idx.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits=5
fold = StratifiedKFold(n_splits=n_splits,shuffle=True)

for trn_idx, val_idx in fold.split(X_Train,Y_Train):

    x_train_idx = X_Train[trn_idx]
    y_train_idx = Y_Train[trn_idx]
    x_valid_idx = X_Train[val_idx]
    y_valid_idx = Y_Train[val_idx]
    
   
    trainset = GetData(TRAIN_DIR, x_train_idx, y_train_idx, Transform_train)
    trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True, num_workers=8)

    validset = GetData(TRAIN_DIR, x_valid_idx, y_valid_idx, Transform_test)
    validloader = DataLoader(validset, batch_size=BATCH, shuffle=True, num_workers=8)    

    for epoch in range(EPOCHS):
        tr_loss = 0.0
        tr_acc = 0.0
        val_loss = 0.0
        val_acc = 0.0
        scheduler.step()
        model.train()
        for i, (images, labels) in enumerate(trainloader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            logits = model(images)
            
            loss = criterion(logits, labels)
            preds_class = torch.argmax(logits, 1)
            
            loss.backward()
            optimizer.step()

            tr_loss += loss.detach().item()
            tr_acc += (preds_class == labels.data).float().mean().data.cpu().numpy()  #считаем accuracy
            
        with torch.no_grad():
            model.eval()
            for i, (images, labels) in enumerate(validloader):
                
                images = images.to(DEVICE)
                labels = labels.to(DEVICE)
                optimizer.zero_grad()
                logits = model(images)
                
                loss = criterion(logits, labels)
                preds_class = torch.argmax(logits, 1)
                
                val_loss += loss.detach().item()
                val_acc += (preds_class == labels.data).float().mean().data.cpu().numpy()  #считаем accuracy
                
        print('Epoch: %d | Train_Loss: %.4f | Train_Acc: %.4f | Valid_Loss: %.4f | Valid_Acc: %.4f '%(epoch, tr_loss / len(trainloader), tr_acc / len(trainloader), val_loss / len(validloader), val_acc / len(validloader)))    

In [ ]:
torch.save(model.state_dict(),'my_model')

In [ ]:
testset = GetData(TEST_DIR, X_Test, None, Transform_test)
testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=8)

In [ ]:
s_ls = []

with torch.no_grad():
    model.eval()
    for image, fname in testloader: 
        image = image.to(DEVICE)
        
        logits = model(image)        
        ps = torch.exp(logits)        
        _, top_class = ps.topk(1, dim=1)
        
        for pred in top_class:
            s_ls.append([fname[0], pred.item()])

In [ ]:
sub = pd.DataFrame.from_records(s_ls, columns=['image_id', 'label'])
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)